In [1]:
%load_ext autoreload
%autoreload 1
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

In [2]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
import timm
from timm import optim, scheduler
import torch
from torchvision import transforms as tfm
from sklearn import metrics as skmet
import matplotlib.pyplot as plt
import json
import transforms as my_transforms

%aimport dataset
ImageData = dataset.ImageData

In [3]:
# artifact_folder = '/zfs/wficai/pda/model_run_artifacts/20220818_justcolor_224x224'
artifact_folder = '/zfs/wficai/pda/model_run_artifacts/aihub_results_SAVE/'

with open(artifact_folder + '/config.json', 'r') as f: 
    cfg = json.load(f)

# put all config variables in scope to avoid the need to laboriously index cfg
for k, v in cfg.items():
    v = f"'{v}'" if type(v)==str else v
    exec(f"{k}={v}")
del cfg

In [4]:
# optionally override settings
view_filter = ['pdaView', 'pdaRelatedView']
mode_filter = ['color', 'color_compare']
device = torch.device('cuda:1')  # you may need 'cuda:0'

In [5]:
tfms = my_transforms.ImageTransforms(res)
tfms_test = tfms.get_transforms(transforms['test'])

In [6]:
df_test = pd.read_csv(f'{artifact_folder}/{out_paths["test"]}')
d_test = ImageData(df_test, transforms = tfms_test, mode_filter = mode_filter, view_filter = view_filter)
dl_test = DataLoader(d_test, batch_size=bs_test, num_workers=num_workers)

print("Number of frames after filtering:", len(d_test.data))

Number of frames after filtering: 11046


In [7]:
# create model
m = timm.create_model(model, pretrained=pretrained, checkpoint_path = f"{artifact_folder}/model_checkpoint.ckpt", num_classes=num_classes, in_chans=3)
m.to(device)           
m.eval()

loss_function = torch.functional.F.binary_cross_entropy_with_logits

target_ls = []
output_ls = []
study_ls = []
video_ls = []
view_ls = []
mode_ls = []
losses = []

for ix, batch in enumerate(dl_test):
    print(f"Batch {ix+1}", end = "\r")
    inputs = batch['img'].to(device)
    targets = batch['trg_type'].to(device).type(torch.float32)
    target_ls.append(targets.cpu().numpy())
    view_ls.append(batch['trg_view'].numpy())
    mode_ls.append(batch['trg_mode'].numpy())
    study_ls += batch['study']
    video_ls += batch['video']

    with torch.no_grad():
        outputs = m(inputs)
        output_ls.append(outputs.cpu().numpy())
        loss = loss_function(outputs.squeeze(), targets)
        losses.append(loss.detach().item())

In [8]:
df_results = pd.DataFrame(dict(
    study = study_ls,
    video = video_ls,
    predicted = np.concatenate(output_ls).squeeze(),
    target = np.concatenate(target_ls), 
    mode = np.concatenate(mode_ls),
    view = np.concatenate(view_ls)
))

df_results['mode'] = df_results['mode'].map(ImageData.inv_mode_map)
df_results.view = df_results.view.map(ImageData.inv_view_map)

df_results.predicted = 1 / (1 + np.exp(-df_results.predicted))

df_results.head(20)

,study,video,predicted,target,mode,view
0,study24,study24_dicom98,0.009909,0.0,color_compare,pdaRelatedView
1,study24,study24_dicom98,0.017972,0.0,color_compare,pdaRelatedView
2,study24,study24_dicom98,0.217672,0.0,color_compare,pdaRelatedView
3,study24,study24_dicom98,0.009441,0.0,color_compare,pdaRelatedView
4,study24,study24_dicom98,0.062039,0.0,color_compare,pdaRelatedView
5,study24,study24_dicom98,0.092517,0.0,color_compare,pdaRelatedView
6,study24,study24_dicom98,0.034904,0.0,color_compare,pdaRelatedView
7,study24,study24_dicom98,0.112240,0.0,color_compare,pdaRelatedView
8,study24,study24_dicom98,0.071711,0.0,color_compare,pdaRelatedView
9,study24,study24_dicom98,0.011965,0.0,color_compare,pdaRelatedView


In [9]:
def compute_metrics(y_true, y_pred, thresh=0.5):
    mets = dict()
    
    y_pred_cls = (y_pred>thresh).astype(int)
    
    mets['num_samples'] = len(y_true)
    mets['roc_auc'] = skmet.roc_auc_score(y_true, y_pred)
    mets['average_precision'] = skmet.average_precision_score(y_true, y_pred)
    mets['accuracy'] = skmet.accuracy_score(y_true, y_pred_cls)
    mets['sensitivity'] = skmet.recall_score(y_true, y_pred_cls)
    mets['specificity'] = skmet.recall_score(y_true, y_pred_cls, pos_label=0)
    
    return mets

# Frame-level results

In [10]:
compute_metrics(df_results.target, df_results.predicted)

{'num_samples': 11046,
 'roc_auc': 0.8115095101936758,
 'average_precision': 0.7785208512112347,
 'accuracy': 0.7328444685859135,
 'sensitivity': 0.7389070527790752,
 'specificity': 0.7290065050266115}

In [11]:
grouped_results = df_results.groupby(['view', 'mode']).apply(lambda dat: compute_metrics(dat.target, dat.predicted))
grouped_results = pd.DataFrame(grouped_results.tolist(), index=grouped_results.index)
grouped_results

num_samples   roc_auc  average_precision  \
view           mode                                                      
pdaRelatedView color                 1446  0.544968           0.410663   
               color_compare         3455  0.769648           0.591618   
pdaView        color                 1546  0.969215           0.924375   
               color_compare         4599  0.857589           0.890833   

                              accuracy  sensitivity  specificity  
view           mode                                               
pdaRelatedView color          0.564315     0.527211     0.573785  
               color_compare  0.711722     0.608991     0.764938  
pdaView        color          0.794955     0.982673     0.728546  
               color_compare  0.780822     0.787526     0.773473

# Clip-level results

### Avg confidence over frames

In [12]:
df_results_clip_avg = df_results.groupby(['study', 'video', 'target', 'view', 'mode'], as_index=False).agg('mean')
display(df_results_clip_avg.head(10))

compute_metrics(df_results_clip_avg.target, df_results_clip_avg.predicted)

,study,video,target,view,mode,predicted
0,study11,study11_dicom68,1.0,pdaRelatedView,color,0.560186
1,study11,study11_dicom72,1.0,pdaView,color,0.914493
2,study11,study11_dicom73,1.0,pdaView,color_compare,0.987371
3,study14,study14_dicom3,0.0,pdaRelatedView,color,0.467666
4,study14,study14_dicom37,0.0,pdaView,color_compare,0.504305
5,study15,study15_dicom54,0.0,pdaRelatedView,color_compare,0.272810
6,study18,study18_dicom41,0.0,pdaRelatedView,color,0.204826
7,study18,study18_dicom63,0.0,pdaRelatedView,color_compare,0.451977
8,study19,study19_dicom17,1.0,pdaRelatedView,color_compare,0.707272
9,study19,study19_dicom18,1.0,pdaView,color_compare,0.840601


{'num_samples': 103,
 'roc_auc': 0.8369175627240144,
 'average_precision': 0.7850428010822897,
 'accuracy': 0.7572815533980582,
 'sensitivity': 0.7741935483870968,
 'specificity': 0.75}

In [13]:
grouped_results_clip_avg = df_results_clip_avg.\
    groupby(['view', 'mode']).\
    apply(lambda dat: compute_metrics(dat.target, dat.predicted, thresh=0.55))
grouped_results_clip_avg = pd.DataFrame(grouped_results_clip_avg.tolist(), index=grouped_results.index)
grouped_results_clip_avg

num_samples   roc_auc  average_precision  \
view           mode                                                      
pdaRelatedView color                   21  0.592593           0.492063   
               color_compare           31  0.869565           0.683036   
pdaView        color                   18  1.000000           1.000000   
               color_compare           33  0.862963           0.882105   

                              accuracy  sensitivity  specificity  
view           mode                                               
pdaRelatedView color          0.714286     0.666667     0.722222  
               color_compare  0.838710     0.625000     0.913043  
pdaView        color          0.888889     1.000000     0.846154  
               color_compare  0.818182     0.733333     0.888889

### Misses

In [14]:
true = df_results_clip_avg.target
pred_cls = (df_results_clip_avg.predicted>0.5).astype(int)

df_results_clip_avg[true != pred_cls]

,study,video,target,view,mode,predicted
4,study14,study14_dicom37,0.0,pdaView,color_compare,0.504305
10,study19,study19_dicom21,1.0,pdaRelatedView,color_compare,0.314879
11,study19,study19_dicom22,1.0,pdaRelatedView,color_compare,0.346817
24,study22,study22_dicom63,1.0,pdaView,color_compare,0.386708
39,study28,study28_dicom12,0.0,pdaView,color,0.763043
40,study28,study28_dicom15,0.0,pdaRelatedView,color,0.528732
41,study28,study28_dicom28,0.0,pdaView,color,0.710325
42,study28,study28_dicom37,0.0,pdaRelatedView,color,0.825942
54,study41,study41_dicom39,0.0,pdaRelatedView,color,0.547446
55,study43,study43_dicom1,1.0,pdaRelatedView,color_compare,0.494346


# Study-level results

In [15]:
df_results_study_avg = df_results_clip_avg.groupby(['study', 'target'], as_index=False).agg('mean')
display(df_results_study_avg)

compute_metrics(df_results_study_avg.target, df_results_study_avg.predicted)

,study,target,predicted
0,study11,1.0,0.820683
1,study14,0.0,0.485985
2,study15,0.0,0.272810
3,study18,0.0,0.328402
4,study19,1.0,0.561275
5,study2,0.0,0.198340
6,study22,1.0,0.761992
7,study24,0.0,0.081092
8,study25,0.0,0.326321
9,study27,0.0,0.246430


{'num_samples': 28,
 'roc_auc': 0.803030303030303,
 'average_precision': 0.7732142857142857,
 'accuracy': 0.7857142857142857,
 'sensitivity': 0.8333333333333334,
 'specificity': 0.7727272727272727}